In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
from difflib import get_close_matches

def find_similar_titles(title, all_titles):
    close = get_close_matches(title, all_titles, n=3, cutoff=0.6)
    return close

In [24]:
from google.colab import files
uploaded = files.upload()


Saving Books.csv to Books (2).csv
Saving Ratings (2).csv to Ratings (2) (3).csv
Saving Users (1).csv to Users (1) (3).csv


In [ ]:
import pandas as pd

books = pd.read_csv("Books.csv", encoding='latin-1')
users = pd.read_csv("Users (1).csv", encoding='latin-1')
ratings = pd.read_csv("Ratings (2).csv", encoding='latin-1')

print("Books:")
print(books.head(2))
print("\nUsers:")
print(users.head(2))
print("\nRatings:")
print(ratings.head(2))

In [ ]:
# Clean column names if needed
books = books[['ISBN', 'Book-Title', 'Book-Author']]
ratings = ratings[['User-ID', 'ISBN', 'Book-Rating']]

# Merge ratings with books
data = pd.merge(ratings, books, on='ISBN')

# Optional: Filter users who rated more than 50 books
popular_users = data['User-ID'].value_counts() > 50
data = data[data['User-ID'].isin(popular_users[popular_users].index)]


In [ ]:
book_pivot = data.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
book_pivot.fillna(0, inplace=True)

from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot.values)

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_sparse)

In [ ]:
def recommend_books(book_name, top_n=5):
    book_pivot = ratings_final.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating')
    book_pivot.fillna(0, inplace=True)

    similarity_scores = cosine_similarity(book_pivot)
    similarity_df = pd.DataFrame(similarity_scores, index=book_pivot.index, columns=book_pivot.index)

    if book_name not in similarity_df.columns:
        close_matches = find_similar_titles(book_name, similarity_df.columns)
        if close_matches:
            print(f"⚠️ Book not found. Did you mean: {', '.join(close_matches)}?")
        else:
            print("❌ Book not found and no close matches.")
        return

    print(f"\n📘 Books similar to: **{book_name}**\n")
    sim_scores = similarity_df[book_name].sort_values(ascending=False)[1:top_n+1]

    for i, title in enumerate(sim_scores.index):
        author = books[books['Book-Title'] == title]['Book-Author'].values[0]
        avg_rating = ratings_final[ratings_final['Book-Title'] == title]['Book-Rating'].mean()
        count_rating = ratings_final[ratings_final['Book-Title'] == title].shape[0]
        print(f"{i+1}. {title} by {author} — ⭐ {avg_rating:.2f} from {count_rating} ratings")

In [ ]:
def popular_books(top_n=10):
    popular_df = data.groupby('Book-Title').agg({'Book-Rating': ['count', 'mean']})
    popular_df.columns = ['rating_count', 'avg_rating']
    popular_df = popular_df.sort_values(by='rating_count', ascending=False).head(top_n)

    print(f"Top {top_n} Popular Books:\n")
    for i, title in enumerate(popular_df.index):
        print(f"{i+1}. {title} | Ratings: {popular_df['rating_count'][i]} | Avg: {round(popular_df['avg_rating'][i], 2)}")

# Example usage:
popular_books(10)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Transpose the pivot for user-user filtering
user_pivot = book_pivot.T
user_sim = cosine_similarity(user_pivot)
user_sim_df = pd.DataFrame(user_sim, index=user_pivot.index, columns=user_pivot.index)

def get_similar_users_books(target_user_id, top_n_users=3):
    if target_user_id not in user_pivot.index:
        return "User not found."

    similar_users = user_sim_df[target_user_id].sort_values(ascending=False)[1:top_n_users+1]
    similar_user_ids = similar_users.index

    books_seen = set(user_pivot.loc[target_user_id][user_pivot.loc[target_user_id] > 0].index)
    recommendations = {}

    for sim_user in similar_user_ids:
        sim_user_books = user_pivot.loc[sim_user][user_pivot.loc[sim_user] > 0]
        for book, rating in sim_user_books.items():
            if book not in books_seen:
                recommendations[book] = recommendations.get(book, 0) + rating

    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"Top recommended books for user {target_user_id}:\n")
    for i, (book, score) in enumerate(sorted_recommendations, 1):
        print(f"{i}. {book} | Score: {round(score, 2)}")

# Example usage:
get_similar_users_books(278700)  # Replace with a real user ID from your data

In [ ]:
def run_recommender():
    while True:
        print("\n📚 Welcome to the Book Recommendation System!")
        print("Choose a method:")
        print("1. Popularity-Based Recommender")
        print("2. User-User Collaborative Filtering")
        print("3. Item-Item Collaborative Filtering")
        print("4. Exit")

        choice = input("Enter your choice (1/2/3/4): ").strip()

        if choice == '1':
            top_n = input("How many top popular books do you want? (e.g., 10): ")
            try:
                top_n = int(top_n)
                popular_books(top_n)
            except:
                print("❌ Please enter a valid number.")

        elif choice == '2':
            user_id = input("Enter your User ID (e.g., 276729): ")
            try:
                user_id = int(user_id)
                get_similar_users_books(user_id)
            except:
                print("❌ Invalid user ID.")

        elif choice == '3':
            book_name = input("Enter a Book Title: ").strip()
            recommend_books(book_name)

        elif choice == '4':
            print("👋 Thank you for using the Book Recommendation System!")
            break

        else:
            print("❌ Invalid choice. Please choose 1, 2, 3, or 4.")

# Run this now:
run_recommender()


📚 Welcome to the Book Recommendation System!
Choose a method:
1. Popularity-Based Recommender
2. User-User Collaborative Filtering
3. Item-Item Collaborative Filtering
4. Exit
